In [1]:
from utils.data_preprocessing import *

In [2]:
base_path = os.path.join('..', 'data', 'dlis_preprocessed')
csv_data = load_csv_files(base_path)

In [3]:
cali_ild_data = create_df_subset(csv_data, ['CALI', 'ILD'])

In [4]:
remove_nan_values(cali_ild_data)

c:\Users\rafae\Documents\College\PIBIC_Imagens\lithofacies-classification-from-image-logs\src\utils\data_preprocessing.py:315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
c:\Users\rafae\Documents\College\PIBIC_Imagens\lithofacies-classification-from-image-logs\src\utils\data_preprocessing.py:315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
c:\Users\rafae\Documents\College\PIBIC_Imagens\lithofacies-classification-from-image-logs\src\utils\data_preprocessing.py:315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

In [5]:
spliced_data = {}
for well, w_dict in cali_ild_data.items():
    well_data = []

    for logical_file, lf_dict in w_dict.items():
        for frame, df in lf_dict.items():
            if 'CALI' in df.columns and 'ILD' in df.columns:
                well_data.append(df)

    try:            
        # Concat all the dfs on the vertical axis
        df_concat = pd.concat(well_data)
    except:
        pass

    spliced_data[well] = df_concat

In [6]:
spliced_dfs_to_csv(spliced_data, '../data/dlis_spliced/CALI')